In [2]:
%matplotlib inline


Sequence Models and Long-Short Term Memory Networks
===================================================

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py

At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding *hidden state* $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.


LSTM's in Pytorch
~~~~~~~~~~~~~~~~~

Before getting to the example, note a few things. Pytorch's LSTM expects
all of its inputs to be 3D tensors. The semantics of the axes of these
tensors is important. The first axis is the sequence itself, the second
indexes instances in the mini-batch, and the third indexes elements of
the input. We haven't discussed mini-batching, so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.



In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

In [3]:
# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print("out:", out, "\nhidden:", hidden, '\n')

out: tensor([[[-0.2682,  0.0304, -0.1526]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.2682,  0.0304, -0.1526]]], grad_fn=<StackBackward>), tensor([[[-1.0766,  0.0972, -0.5498]]], grad_fn=<StackBackward>)) 

out: tensor([[[-0.5370,  0.0346, -0.1958]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.5370,  0.0346, -0.1958]]], grad_fn=<StackBackward>), tensor([[[-1.1552,  0.1214, -0.2974]]], grad_fn=<StackBackward>)) 

out: tensor([[[-0.3947,  0.0391, -0.1217]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.3947,  0.0391, -0.1217]]], grad_fn=<StackBackward>), tensor([[[-1.0727,  0.1104, -0.2179]]], grad_fn=<StackBackward>)) 

out: tensor([[[-0.1854,  0.0740, -0.0979]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.1854,  0.0740, -0.0979]]], grad_fn=<StackBackward>), tensor([[[-1.0530,  0.1836, -0.1731]]], grad_fn=<StackBackward>)) 

out: tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>)

In [4]:
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(inputs)
print(out)
print(hidden)

tensor([[[-0.5525,  0.6355, -0.3968]],

        [[-0.6571, -1.6428,  0.9803]],

        [[-0.0421, -0.8206,  0.3133]],

        [[-1.1352,  0.3773, -0.2824]],

        [[-2.5667, -1.4303,  0.5009]]])
tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


Example: An LSTM for Part-of-Speech Tagging
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In this section, we will use an LSTM to get part of speech tags. We will
not use Viterbi or Forward-Backward or anything like that, but as a
(challenging) exercise to the reader, think about how Viterbi could be
used after you have seen what is going on.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.


Prepare data:



In [5]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


Create the model:



In [6]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [7]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
tensor([[-0.0462, -4.0106, -3.6096],
        [-4.8205, -0.0286, -3.9045],
        [-3.7876, -4.1355, -0.0394],
        [-0.0185, -4.7874, -4.6013],
        [-5.7881, -0.0186, -4.1778]])


Exercise: Augmenting the LSTM part-of-speech tagger with character-level features
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In the example above, each word had an embedding, which served as the
inputs to our sequence model. Let's augment the word embeddings with a
representation derived from the characters of the word. We expect that
this should help significantly, since character-level information like
affixes have a large bearing on part-of-speech. For example, words with
the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of
word $w$. Let $x_w$ be the word embedding as before. Then
the input to our sequence model is the concatenation of $x_w$ and
$c_w$. So if $x_w$ has dimension 5, and $c_w$
dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the
characters of a word, and let $c_w$ be the final hidden state of
this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.




In [8]:
character_to_ix = {}
for sent, _ in training_data:
    for word in sent:
        for char in word:
            if char not in character_to_ix.keys():
                character_to_ix[char]=len(character_to_ix)

In [9]:
def word_to_char_ix_tensor(word):
    indices = []
    for char in word:
        indices.append(character_to_ix[char])
    return torch.tensor(indices, dtype=torch.long)

In [10]:
char_dim = 5

In [11]:
class LSTMCharTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, char_emb_dim, char_hidden_dim, vocab_size, char_size, tagset_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.char_hidden_dim = char_hidden_dim
        self.embedding_dim = embedding_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.char_embeddings = nn.Embedding(char_size, char_emb_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.char_lstm = nn.LSTM(char_emb_dim, char_hidden_dim)
        self.lstm = nn.LSTM(embedding_dim+char_emb_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(prepare_sequence(sentence,word_to_ix))
        res = []
        for word in sentence:
            char_emb = self.char_embeddings(word_to_char_ix_tensor(word))
            hidden, _ = self.char_lstm(char_emb.view(len(word),1,-1))
            res.append(hidden[-1])
        res = torch.cat(res)
        vec = torch.cat((embeds, res),dim=1)
       
        lstm_out, _ = self.lstm(vec.view(len(sentence), 1, -1))
       
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [12]:
model = LSTMCharTagger(EMBEDDING_DIM, HIDDEN_DIM, char_dim, char_dim, len(word_to_ix), len(character_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [13]:

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = training_data[0][0]
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = sentence
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = training_data[0][0]
    tag_scores = model(inputs)
    print(prepare_sequence(training_data[0][1], tag_to_ix))

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.2465, -1.4033, -0.7621],
        [-1.2876, -1.3773, -0.7512],
        [-1.2925, -1.4044, -0.7342],
        [-1.2500, -1.4602, -0.7313],
        [-1.3286, -1.4408, -0.6963]])
tensor([0, 1, 2, 0, 1])
tensor([[-8.0109e-05, -9.4504e+00, -1.3301e+01],
        [-1.2197e+01, -1.0777e-04, -9.1805e+00],
        [-1.0943e+01, -8.4406e+00, -2.3365e-04],
        [-3.1471e-05, -1.0417e+01, -1.3120e+01],
        [-1.1806e+01, -9.5367e-06, -1.2933e+01]])


In [14]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\lih09\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [15]:
wsj = list(nltk.corpus.treebank.tagged_sents())

In [16]:
wsj[0]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

POS tagger의 정의는 다음과 같다.

|Number|Tag|Description|
|:-:|:-:|:-:|
|1|CC|Coordinating conjunction|
|2|CD|Cardinal number|
|3|DT|Determiner|
|4|EX|Existential there|
|5|FW|Foreign word|
|6|IN|Preposition or subordinating conjunction|
|7|JJ|Adjective|
|8|JJR|Adjective, comparative|
|9|JJS|Adjective, superlative|
|10|LS|List item marker|
|11|MD|Modal|
|12|NN|Noun, singular or mass|
|13|NNS|Noun, plural|
|14|NNP|Proper noun, singular|
|15|NNPS|Proper noun, plural|
|16|PDT|Predeterminer|
|17|POS|Possessive ending|
|18|PRP|Personal pronoun|
|19|PRP_dollar|Possessive pronoun|
|20|RB|Adverb|
|21|RBR|Adverb, comparative|
|22|RBS|Adverb, superlative|
|23|RP|Particle|
|24|SYM|Symbol|
|25|TO|to|
|26|UH|Interjection|
|27|VB|Verb, base form|
|28|VBD|Verb, past tense|
|29|VBG|Verb, gerund or present participle|
|30|VBN|Verb, past participle|
|31|VBP|Verb, non-3rd person singular present|
|32|VBZ|Verb, 3rd person singular present|
|33|WDT|Wh-determiner|
|34|WP|Wh-pronoun|
|35|WP_dollar|Possessive wh-pronoun|
|36|WRB|Wh-adverb|


In [42]:
data = []
for example in wsj:
    sent = []
    tt = []
    for word, vv in example:
        sent.append(word)
        tt.append(vv)
    data.append((sent,tt))

In [43]:
data[0]

(['Pierre',
  'Vinken',
  ',',
  '61',
  'years',
  'old',
  ',',
  'will',
  'join',
  'the',
  'board',
  'as',
  'a',
  'nonexecutive',
  'director',
  'Nov.',
  '29',
  '.'],
 ['NNP',
  'NNP',
  ',',
  'CD',
  'NNS',
  'JJ',
  ',',
  'MD',
  'VB',
  'DT',
  'NN',
  'IN',
  'DT',
  'JJ',
  'NN',
  'NNP',
  'CD',
  '.'])

In [20]:
character_to_ix = {}
for sent, _ in data:
    for word in sent:
        for char in word:
            if char not in character_to_ix.keys():
                character_to_ix[char]=len(character_to_ix)

In [37]:
word_to_ix = {}
tag_to_ix = {}
ix_to_tag = {}
for sent, tags in data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)
            ix_to_tag[len(tag_to_ix)-1]=tag
            
print(word_to_ix)
print(tag_to_ix)
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 64
HIDDEN_DIM = 64

{'Pierre': 0, 'Vinken': 1, ',': 2, '61': 3, 'years': 4, 'old': 5, 'will': 6, 'join': 7, 'the': 8, 'board': 9, 'as': 10, 'a': 11, 'nonexecutive': 12, 'director': 13, 'Nov.': 14, '29': 15, '.': 16, 'Mr.': 17, 'is': 18, 'chairman': 19, 'of': 20, 'Elsevier': 21, 'N.V.': 22, 'Dutch': 23, 'publishing': 24, 'group': 25, 'Rudolph': 26, 'Agnew': 27, '55': 28, 'and': 29, 'former': 30, 'Consolidated': 31, 'Gold': 32, 'Fields': 33, 'PLC': 34, 'was': 35, 'named': 36, '*-1': 37, 'this': 38, 'British': 39, 'industrial': 40, 'conglomerate': 41, 'A': 42, 'form': 43, 'asbestos': 44, 'once': 45, 'used': 46, '*': 47, 'to': 48, 'make': 49, 'Kent': 50, 'cigarette': 51, 'filters': 52, 'has': 53, 'caused': 54, 'high': 55, 'percentage': 56, 'cancer': 57, 'deaths': 58, 'among': 59, 'workers': 60, 'exposed': 61, 'it': 62, 'more': 63, 'than': 64, '30': 65, 'ago': 66, 'researchers': 67, 'reported': 68, '0': 69, '*T*-1': 70, 'The': 71, 'fiber': 72, 'crocidolite': 73, 'unusually': 74, 'resilient': 75, 'enters': 76, 


{'NNP': 0, ',': 1, 'CD': 2, 'NNS': 3, 'JJ': 4, 'MD': 5, 'VB': 6, 'DT': 7, 'NN': 8, 'IN': 9, '.': 10, 'VBZ': 11, 'VBG': 12, 'CC': 13, 'VBD': 14, 'VBN': 15, '-NONE-': 16, 'RB': 17, 'TO': 18, 'PRP': 19, 'RBR': 20, 'WDT': 21, 'VBP': 22, 'RP': 23, 'PRP$': 24, 'JJS': 25, 'POS': 26, '``': 27, 'EX': 28, "''": 29, 'WP': 30, ':': 31, 'JJR': 32, 'WRB': 33, '$': 34, 'NNPS': 35, 'WP$': 36, '-LRB-': 37, '-RRB-': 38, 'PDT': 39, 'RBS': 40, 'FW': 41, 'UH': 42, 'SYM': 43, 'LS': 44, '#': 45}


In [22]:
EMBEDDING_DIM = 64
HIDDEN_DIM = 128
CHAR_EMB_DIM = 64
CHAR_HIDDEN_DIM = 128 

In [23]:
class LSTMCharTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, char_emb_dim, char_hidden_dim, vocab_size, char_size, tagset_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.char_hidden_dim = char_hidden_dim
        self.embedding_dim = embedding_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.char_embeddings = nn.Embedding(char_size, char_emb_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.char_lstm = nn.LSTM(char_emb_dim, char_hidden_dim)
        self.lstm = nn.LSTM(embedding_dim+char_emb_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(prepare_sequence(sentence,word_to_ix))
        res = []
        for word in sentence:
            char_emb = self.char_embeddings(word_to_char_ix_tensor(word))
            hidden, _ = self.char_lstm(char_emb.view(len(word),1,-1))
            res.append(hidden[-1])
        res = torch.cat(res)
        vec = torch.cat((embeds, res),dim=1)
       
        lstm_out, _ = self.lstm(vec.view(len(sentence), 1, -1))
       
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [24]:
model = LSTMCharTagger(EMBEDDING_DIM, HIDDEN_DIM, char_dim, char_dim, len(word_to_ix), len(character_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [57]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = data[0][0]
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(3):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = sentence
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

        

tensor([[-3.7344, -3.9005, -3.7731, -3.8801, -3.8895, -3.8074, -4.0374, -3.8008,
         -3.8104, -3.7294, -3.8751, -3.7962, -3.8265, -3.7550, -3.7460, -3.8504,
         -3.8253, -3.7725, -3.8026, -3.9645, -3.8119, -3.8607, -3.9561, -3.7763,
         -3.7812, -3.9031, -3.8919, -3.8813, -3.7572, -3.8866, -3.8622, -3.7470,
         -3.7864, -3.7949, -3.8941, -3.8938, -3.7851, -3.8512, -3.7881, -3.8103,
         -3.8257, -3.8744, -3.8127, -3.7259, -3.7743, -3.9094],
        [-3.8168, -3.9156, -3.7655, -3.8499, -3.8481, -3.7835, -4.0177, -3.7456,
         -3.7312, -3.8146, -3.9228, -3.7400, -3.9654, -3.7770, -3.7876, -3.9606,
         -3.8277, -3.8645, -3.7323, -3.8638, -3.7553, -3.8553, -3.9267, -3.7693,
         -3.7995, -3.8569, -3.8479, -3.8609, -3.6865, -3.9427, -3.9638, -3.7663,
         -3.7664, -3.7257, -3.8755, -3.9674, -3.8785, -3.7826, -3.7363, -3.8389,
         -3.8302, -3.8829, -3.8833, -3.6931, -3.7382, -3.9066],
        [-3.7725, -3.8899, -3.6758, -3.9133, -3.7687, -3.7827,

         -3.7548, -3.8383, -3.7017, -3.8051, -3.9420, -3.7751]])


TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [59]:

# See what the scores are after training
with torch.no_grad():
    inputs = data[0][0]
    tag_scores = model(inputs)
    print(prepare_sequence(data[0][1], tag_to_ix))

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([ 0,  0,  1,  2,  3,  4,  1,  5,  6,  7,  8,  9,  7,  4,  8,  0,  2, 10])
tensor([[-3.5678e+00, -4.9737e+01, -4.3227e+00, -1.4063e+00, -1.4275e+00,
         -3.5408e+01, -5.5141e+00, -1.4202e+01, -1.0534e+00, -4.8636e+00,
         -6.5475e+01, -5.6267e+00, -2.8334e+00, -4.4211e+01, -3.5878e+00,
         -6.3130e+00, -1.6342e+01, -3.9241e+00, -4.2419e+01, -2.1314e+01,
         -1.6051e+01, -5.0322e+01, -1.1362e+01, -2.5206e+01, -3.1555e+01,
         -3.5585e+01, -4.5091e+01, -4.9898e+01, -5.3415e+01, -5.8657e+01,
         -4.2977e+01, -2.7701e+01, -6.7174e+00, -3.9132e+01, -4.0839e+01,
         -8.5999e+00, -6.5209e+01, -6.6028e+01, -7.3303e+01, -6.5591e+01,
         -8.3057e+01, -5.4793e+01, -4.1426e+01, -4.9163e+01, -4.7126e+01,
         -8.2382e+01],
        [-3.3978e+00, -7.8052e+01, -2.1189e+00, -6.5698e+00, -8.9828e-01,
         -1.5113e+01, -1.9816e+00, -1.7752e+01, -1.3884e+00, -5.6294e+00,
         -4.9994e+01, -3.2028e+00, -5.6290e+00, -3.7419e+01, -8.5601e+00,
        

         -1.5653e+02]])


In [61]:
torch.save(model.state_dict(), 'char_lstm_3pochs.pkl')

In [26]:
model.load_state_dict(torch.load('char_lstm_3pochs.pkl'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [50]:
def pos_sampling(sent):
    ans = ''
    exam = sent.split()
    for i in model(exam):
        ans+=ix_to_tag[int(torch.argmax(i))]+' '
    print(ans)

In [51]:
pos_sampling('nice to meet you')

VBD TO VB PRP 


In [57]:
pos_sampling('we need to study hard') # 한계가 있다! study는 명사 동사 모두 가능

VBZ JJ TO NNP NNP 


In [60]:
pos_sampling('I enjoy watching movies')

NNP NN PRP NN 


POS tagger의 정의는 다음과 같다.

|Number|Tag|Description|
|:-:|:-:|:-:|
|1|CC|Coordinating conjunction|
|2|CD|Cardinal number|
|3|DT|Determiner|
|4|EX|Existential there|
|5|FW|Foreign word|
|6|IN|Preposition or subordinating conjunction|
|7|JJ|Adjective|
|8|JJR|Adjective, comparative|
|9|JJS|Adjective, superlative|
|10|LS|List item marker|
|11|MD|Modal|
|12|NN|Noun, singular or mass|
|13|NNS|Noun, plural|
|14|NNP|Proper noun, singular|
|15|NNPS|Proper noun, plural|
|16|PDT|Predeterminer|
|17|POS|Possessive ending|
|18|PRP|Personal pronoun|
|19|PRP_dollar|Possessive pronoun|
|20|RB|Adverb|
|21|RBR|Adverb, comparative|
|22|RBS|Adverb, superlative|
|23|RP|Particle|
|24|SYM|Symbol|
|25|TO|to|
|26|UH|Interjection|
|27|VB|Verb, base form|
|28|VBD|Verb, past tense|
|29|VBG|Verb, gerund or present participle|
|30|VBN|Verb, past participle|
|31|VBP|Verb, non-3rd person singular present|
|32|VBZ|Verb, 3rd person singular present|
|33|WDT|Wh-determiner|
|34|WP|Wh-pronoun|
|35|WP_dollar|Possessive wh-pronoun|
|36|WRB|Wh-adverb|
